In [7]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from src.models.modules import *
from src.models.loss import L1_clamp_loss
from dataclasses import dataclass
import torch

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    num_inds: int = 32
    dim_hidden: int = 128
    num_heads: int = 4
    ln: bool = True

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.enc = nn.Sequential(
            ISAB(config.dim_context, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            ISAB(config.dim_hidden, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            ISAB(config.dim_hidden, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln)
        )
        self.input_proj = nn.Sequential(nn.Linear(config.dim_input, config.dim_hidden), nn.ReLU())
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.dec = nn.Sequential(
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
        )
        self.regr = nn.Sequential(
            nn.ReLU(),
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        y = self.enc(context)           # [batch_size, context_size, dim_hidden]
        x = self.input_proj(x)          # [batch_size, num_outputs, dim_hidden]
        x = x.repeat(1, y.shape[1], 1)  # [batch_size, context_size, dim_hidden]
        y = self.cross(x, y)            # [batch_size, context_size, dim_hidden]
        y = self.dec(y)                 # [batch_size, num_outputs, dim_hidden]
        y = self.regr(y)                # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_clamp_loss(y, labels, self.config.delta)
        return {'loss': loss, 'logits': y}

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [9]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [10]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_11_21_layer_norm'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_11_21_layer_norm-2024-11-21-20-10-38


In [11]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0,
    save_total_limit=3,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [12]:
trainer.train()
train_dataset.close()
val_dataset.close()

  0%|          | 0/120000 [00:00<?, ?it/s]

{'loss': 0.1034, 'grad_norm': 0.0, 'learning_rate': 4.999583333333333e-05, 'epoch': 0.0}
{'loss': 0.1029, 'grad_norm': 0.0, 'learning_rate': 4.999166666666667e-05, 'epoch': 0.0}
{'loss': 0.1033, 'grad_norm': 0.0, 'learning_rate': 4.99875e-05, 'epoch': 0.0}
{'loss': 0.1021, 'grad_norm': 0.0, 'learning_rate': 4.998333333333334e-05, 'epoch': 0.0}
{'loss': 0.1048, 'grad_norm': 0.0, 'learning_rate': 4.997916666666667e-05, 'epoch': 0.0}
{'loss': 0.1022, 'grad_norm': 0.0, 'learning_rate': 4.9975e-05, 'epoch': 0.0}
{'loss': 0.105, 'grad_norm': 0.0, 'learning_rate': 4.997083333333333e-05, 'epoch': 0.0}
{'loss': 0.1066, 'grad_norm': 0.0, 'learning_rate': 4.996666666666667e-05, 'epoch': 0.01}
{'loss': 0.1049, 'grad_norm': 0.0, 'learning_rate': 4.99625e-05, 'epoch': 0.01}
{'loss': 0.104, 'grad_norm': 0.0, 'learning_rate': 4.995833333333333e-05, 'epoch': 0.01}
{'loss': 0.1035, 'grad_norm': 0.0, 'learning_rate': 4.995416666666667e-05, 'epoch': 0.01}
{'loss': 0.1037, 'grad_norm': 0.0, 'learning_rate'

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.10360348969697952, 'eval_runtime': 152.2155, 'eval_samples_per_second': 1313.927, 'eval_steps_per_second': 32.848, 'epoch': 1.0}
{'loss': 0.1043, 'grad_norm': 0.0, 'learning_rate': 4.374583333333334e-05, 'epoch': 1.0}
{'loss': 0.104, 'grad_norm': 0.0, 'learning_rate': 4.374166666666667e-05, 'epoch': 1.0}
{'loss': 0.1036, 'grad_norm': 0.0, 'learning_rate': 4.3737500000000006e-05, 'epoch': 1.0}
{'loss': 0.1028, 'grad_norm': 0.0, 'learning_rate': 4.373333333333334e-05, 'epoch': 1.0}
{'loss': 0.103, 'grad_norm': 0.0, 'learning_rate': 4.372916666666667e-05, 'epoch': 1.0}
{'loss': 0.1057, 'grad_norm': 0.0, 'learning_rate': 4.3725000000000006e-05, 'epoch': 1.0}
{'loss': 0.1027, 'grad_norm': 0.0, 'learning_rate': 4.372083333333333e-05, 'epoch': 1.0}
{'loss': 0.1041, 'grad_norm': 0.0, 'learning_rate': 4.371666666666667e-05, 'epoch': 1.01}
{'loss': 0.1042, 'grad_norm': 0.0, 'learning_rate': 4.37125e-05, 'epoch': 1.01}
{'loss': 0.1042, 'grad_norm': 0.0, 'learning_rate': 4.37083333

  0%|          | 0/5000 [00:00<?, ?it/s]

...

{'eval_loss': 0.10360348969697952, 'eval_runtime': 167.4289, 'eval_samples_per_second': 1194.537, 'eval_steps_per_second': 29.863, 'epoch': 6.0}
{'loss': 0.1046, 'grad_norm': 0.0, 'learning_rate': 1.2495833333333335e-05, 'epoch': 6.0}
{'loss': 0.1032, 'grad_norm': 0.0, 'learning_rate': 1.2491666666666668e-05, 'epoch': 6.0}
{'loss': 0.1055, 'grad_norm': 0.0, 'learning_rate': 1.24875e-05, 'epoch': 6.0}
{'loss': 0.1033, 'grad_norm': 0.0, 'learning_rate': 1.2483333333333335e-05, 'epoch': 6.0}
{'loss': 0.1031, 'grad_norm': 0.0, 'learning_rate': 1.2479166666666667e-05, 'epoch': 6.0}
{'loss': 0.1037, 'grad_norm': 0.0, 'learning_rate': 1.2475e-05, 'epoch': 6.0}
{'loss': 0.103, 'grad_norm': 0.0, 'learning_rate': 1.2470833333333334e-05, 'epoch': 6.0}
{'loss': 0.1027, 'grad_norm': 0.0, 'learning_rate': 1.2466666666666667e-05, 'epoch': 6.01}
{'loss': 0.104, 'grad_norm': 0.0, 'learning_rate': 1.2462500000000001e-05, 'epoch': 6.01}
{'loss': 0.1031, 'grad_norm': 0.0, 'learning_rate': 1.24583333333333

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.10360348969697952, 'eval_runtime': 171.6278, 'eval_samples_per_second': 1165.313, 'eval_steps_per_second': 29.133, 'epoch': 7.0}
{'loss': 0.1046, 'grad_norm': 0.0, 'learning_rate': 6.245833333333334e-06, 'epoch': 7.0}
{'loss': 0.1039, 'grad_norm': 0.0, 'learning_rate': 6.241666666666667e-06, 'epoch': 7.0}
{'loss': 0.1029, 'grad_norm': 0.0, 'learning_rate': 6.2375e-06, 'epoch': 7.0}
{'loss': 0.1036, 'grad_norm': 0.0, 'learning_rate': 6.2333333333333335e-06, 'epoch': 7.0}
{'loss': 0.1025, 'grad_norm': 0.0, 'learning_rate': 6.229166666666667e-06, 'epoch': 7.0}
{'loss': 0.1036, 'grad_norm': 0.0, 'learning_rate': 6.2250000000000005e-06, 'epoch': 7.0}
{'loss': 0.1019, 'grad_norm': 0.0, 'learning_rate': 6.220833333333333e-06, 'epoch': 7.0}
{'loss': 0.1047, 'grad_norm': 0.0, 'learning_rate': 6.2166666666666676e-06, 'epoch': 7.01}
{'loss': 0.1035, 'grad_norm': 0.0, 'learning_rate': 6.2125e-06, 'epoch': 7.01}
{'loss': 0.1034, 'grad_norm': 0.0, 'learning_rate': 6.208333333333334e-

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.10360348969697952, 'eval_runtime': 166.158, 'eval_samples_per_second': 1203.674, 'eval_steps_per_second': 30.092, 'epoch': 8.0}
{'train_runtime': 9013.2123, 'train_samples_per_second': 532.552, 'train_steps_per_second': 13.314, 'train_loss': 0.10369419160286586, 'epoch': 8.0}


In [13]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)

In [14]:
from src.visualization.generate_mesh import generate_mesh
from src.data.load_data import get_data_dir

obj_path = get_data_dir() / 'intermediate' / 'bunny' / 'stanford-bunny.obj'
infered_obj_path = result_dir / f"bunny-{current_date}-generalization.obj"
generate_mesh(model, obj_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

ValueError: Surface level must be within volume data range.

In [ ]:
hdf5_path = get_data_dir() / 'processed' / 'bunny' / 'stanford-bunny_train.hdf5'
infered_obj_path = result_dir / f"{current_date}-train.obj"
generate_mesh(model, hdf5_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]